In [ ]:
import ROOT
import numpy as np

# silence most of the roofit message (bug in ROOT saturate jupyter stream)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.NumIntegration)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Fitting)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Minimization)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.InputArguments)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Eval)
ROOT.RooMsgService.instance().setGlobalKillBelow(ROOT.RooFit.ERROR)

ROOT.RooStats.AsymptoticCalculator.SetPrintLevel(-1)

ROOT.enableJSVis()

# Limits
Set upper limit is not so different from computing the p-value of the background hypothesis as in the discovery. What you try to do is to exclude signal+background hypothesis with differenet value for the signal (cross section). Usually you are interested to get the set of hypothesis exlucuded at least at 95% CL (and using CLs).

As dataset we are using real data which contains signal, so we expect difference between the observed and expected (under the hypothesis there is no signal) limit.

Remember: this is for a fixed $m_H$

In [ ]:
f = ROOT.TFile.Open('../data/workspace.root')
ws = f.Get("ws")
data = ws.data('data_binned')

# we are trying to exclude xsection values doing a scan
SCAN_MIN = 0
SCAN_MAX = 0.3
SCAN_NSTEPS = 50

In [ ]:
# use the asymptotic calculator
calculator = ROOT.RooStats.AsymptoticCalculator(ws.data('data_binned'),
                                                ws.obj('b_model_config'),
                                                ws.obj('sb_model_config'), False)
calculator.SetOneSided(True)  # the kind of statistical test is defined on the asym calculator

# this is the main object doing the "hypothesis inversion": it runs the scan on the cross section
# finding the ones excluded at least at 95% CL
inverter = ROOT.RooStats.HypoTestInverter(calculator)
inverter.SetConfidenceLevel(0.95)
inverter.SetFixedScan(SCAN_NSTEPS, SCAN_MIN, SCAN_MAX)  # there is also a smarter way without a scan
inverter.UseCLs(True)

result = inverter.GetInterval()  # get the final result
print 'upper limit (obs)', result.UpperLimit()
print 'upper limit (exp)', result.GetExpectedUpperLimit()

# do a plot of the CLs vs cross section
plot = ROOT.RooStats.HypoTestInverterPlot("plot", "plot", result)
canvas = ROOT.TCanvas()
plot.Draw("CLb 2CL")
canvas.Draw()

## StandardHypoTestInvDemo

This is a popular macro to run exclusions. It saves the result in a file.  In the output file you will find a `HypoTestInverterResult` object (as the `result` variable in the example above). The good thig of this class it is mergiable. The usual workflow with heavy workspace is to submit the macro for one single cross section pointa and $m_H$ (or any other signal parameter) (you need to tweak the macro to change the output filename) and then to merge the result to get the final CLs plot.

In [ ]:
# if having problem here (too much output crash jupyter), open ROOT (root -l) and just execute these lines
# .L ../StandardHypoTestInvDemo.C
# StandardHypoTestInvDemo("test.root", "ws", "sb_model_config", "", "data_binned", 2, 3, True, 50, 0, 0.3)
#
# or (probably better) copy this cell in a python standalone program (.py)
ROOT.gROOT.ProcessLine('.L ../StandardHypoTestInvDemo.C')

gr_observed = ROOT.TGraph()

for imH, mH_value in enumerate(np.linspace(110, 130, 20)):
    print mH_value
    ws.var('mH').setVal(mH_value)
    ws.writeToFile('test.root')
    ROOT.StandardHypoTestInvDemo("test.root",
                                 "ws",
                                 "sb_model_config", "",
                                 "data_binned",
                                 2,
                                 3,
                                 True,
                                 SCAN_NSTEPS, SCAN_MIN, SCAN_MAX)
    # file produced by the macro
    f = ROOT.TFile.Open("Asym_CLs_grid_ts3_test.root")
    r = f.Get("result_xsection_x_br")
    gr_observed.SetPoint(imH, mH_value, r.UpperLimit())

In [ ]:
canvas = ROOT.TCanvas()
gr_observed.Draw("APL")
gr_observed.GetYaxis().SetTitle('excluded cross section @ 95% CL')
gr_observed.GetXaxis().SetTitle('m_{H} [GeV]')

canvas.Draw()

# remember that we have defined the SM to have signal cross-section = 0.1

*** Exercize ***: finish the exercize adding the expected with the +/- 1, 2 sigma bands to get the brazilian plot. Use `r.GetExpectedUpperLimit()`, `r.GetExpectedUpperLimit(+1)`, `GetExpectedUpperLimit(-1)`, ... Best way is to use `TGraphAsymmErrors` plotting with `E3` option.